## Load Required Packages

In [1]:
import nltk
import pandas as pd
from os import listdir
from os.path import isfile, join
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
import numpy as np

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

The data used here is from [Apache SpamAssassin](data sourcehttps://spamassassin.apache.org/old/publiccorpus/) and contains a few thousand classified "spam" and "ham" emails.

The uncompressed data is a bit large for github, so we've preprocessed the data and stored it in a single CSV file [here](https://raw.githubusercontent.com/plb2018/data620/master/Week10/spam_n_ham.csv) to make life easier.  The compressed data has been archived [here](https://github.com/plb2018/data620/tree/master/Week10/raw_data) for reproducibility.

In [2]:

PATH = "C:/Users/Paul/OneDrive - CUNY School of Professional Studies/CUNY/DATA 620/Week10/"
spam_folders = ['easy_ham/','spam/','easy_ham_2/','hard_ham/','spam_2/']

cols = ['body','source','isSpam']
df = pd.DataFrame(columns = cols)

for folder in spam_folders:
    files = [f for f in listdir(PATH+folder) if isfile(join(PATH+folder, f))]
    
    for f in files:
        data = open(PATH+folder+f,'rb')
          
        newData = {'body':data.read(),
                  'source':folder,
                  'isSpam':0}
        
        df = df.append(newData,ignore_index=True)
        data.close()

#encode the spam as spam
df['isSpam'][(df['source'] == 'spam/') | (df['source'] == 'spam_2/')] = 1   

#df.to_csv("spam_n_ham.csv",sep='|')


In [3]:
df
df.shape


(9348, 3)

In [4]:
 def cleanData(data):
    data=str(data)
    data = data.lower()
    data=data.replace('{html}',"") 
    clnr = re.compile('<.*?>')
    cleantext = re.sub(clnr, '', data)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)



In [ ]:
df['body']=df['body'].map(lambda s:cleanData(s)) 

In [ ]:
df.head(20)

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/plb2018/data620/master/Week10/spam_n_ham.csv",sep='|',index_col=0)

In [ ]:
df.head(20)

In [ ]:

tts_idx = np.random.rand(df.shape[0]) < 0.75

train = df[tts_idx].sample(frac=1) 
test = df[~tts_idx].sample(frac=1) 


In [ ]:
df.isSpam.sum() /df.shape[0]
train.isSpam.sum()/train.shape[0]
test.isSpam.sum()/test.shape[0]

test.head(25)

In [ ]:
#spam = ' '.join(list(df[df['isSpam'] == 1]['body']))
#spam_wc = WordCloud(width = 512, height=512).generate(spam)
#plt.figure(figsize = (10,8), facecolor = 'w')
#plt.imshow(spam_wc)


In [ ]:
spam_finder_tfidf = SpamClassifier(train,'tf-idf')